<a href="https://colab.research.google.com/github/aniketinamdar/IH-Project-Question-Answering-Model-/blob/main/CosineSimilarity%26NLP%26BagOfWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Downloading dependencies and importing libraries

In [ ]:
import pandas as pd
import json
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
import numpy
from gensim import corpora
import pprint

##Reading json file

In [ ]:
# with open(r"qna.json","r") as read_file:
#   qna = json.load(read_file)
df = pd.read_json (r'qna.json')

In [ ]:
qna

{'answer': {'0': 'CGPA is seen on the grade card. The CGPA count can be verified by referring to the passing    criteria    mentioned    in    Exam    Ordinance    2020(https://mitwpu.edu.in/examination/images/documents/Examination Ordinances 2020.pdf)',
  '1': ' GPA and CGPA are not mentioned on backlog grade cards. GPA, CGPA will be highlighted on regular trimester grade card only.',
  '2': ' Grade card shows first attempt GPA and CGPA only. Hence, even if you have cleared backlog, improved CGPA for that trimester will not be seen on grade card.'},
 'question': {'0': 'What if I need to know my correct CGPA?',
  '1': 'I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?',
  '2': 'I have cleared my backlog, but the result on ERP is not updated .It still shows earlier CGPA.As a result, it is affecting my CGPA of next trimesters also. How do I get it corrected?'}}

##Converting json file to csv file


In [ ]:
df.to_csv(r'fina_qna.csv',index=None)

In [ ]:
df

,question,answer
0,What if I need to know my correct CGPA?,CGPA is seen on the grade card. The CGPA count...
1,I have cleared my backlog but the result sheet...,GPA and CGPA are not mentioned on backlog gra...
2,"I have cleared my backlog, but the result on E...",Grade card shows first attempt GPA and CGPA o...
3,What do I do if I have not received my grade c...,Grade cards should be collected from the stud...
4,"My name,Father's Name,Mother's name is incorre...",Name correction on grade cards/PDC needs to b...
5,How can I get a new copy of corrected grade ca...,Once the required corrections are done in ERP...
6,From where should I collect the corrected grad...,You have to collect the corrected grade cards...
7,In how many days will I get the corrected grad...,Processing of correction on grade cards will ...


##Cleaning the data

In [ ]:
def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    return sentence
                    
def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["question"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        cleaned=clean_sentence(row["question"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords)

['need know correct cgpa', 'cleared backlog result sheet gpa cgpa', 'cleared backlog result erp updated shows earlier cgpaas result affecting cgpa trimesters corrected', 'received grade card', 'namefathers namemothers incorrect grade card corrected', 'new copy corrected grade cards', 'collect corrected grade cardspdc', 'days corrected grade cards']


['what if i need to know my correct cgpa', 'i have cleared my backlog but the result sheet does not show gpa and cgpa why', 'i have cleared my backlog but the result on erp is not updated it still shows earlier cgpaas a result it is affecting my cgpa of next trimesters also how do i get it corrected', 'what do i do if i have not received my grade card', 'my namefathers namemothers name is incorrect on grade card how do i get it corrected', 'how can i get a new copy of corrected grade cards', 'from where should i collect the corrected grade cardspdc', 'in how many days will i get the corrected grade cards']


In [ ]:


sentences=cleaned_sentences_with_stopwords
sentence_words = [[word for word in document.split() ]
         for document in sentences]



dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)


bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

0  :  cgpa
1  :  correct
2  :  i
3  :  if
4  :  know
5  :  my
6  :  need
7  :  to
8  :  what
9  :  and
10  :  backlog
11  :  but
12  :  cleared
13  :  does
14  :  gpa
15  :  have
16  :  not
17  :  result
18  :  sheet
19  :  show
20  :  the
21  :  why
22  :  a
23  :  affecting
24  :  also
25  :  cgpaas
26  :  corrected
27  :  do
28  :  earlier
29  :  erp
30  :  get
31  :  how
32  :  is
33  :  it
34  :  next
35  :  of
36  :  on
37  :  shows
38  :  still
39  :  trimesters
40  :  updated
41  :  card
42  :  grade
43  :  received
44  :  incorrect
45  :  name
46  :  namefathers
47  :  namemothers
48  :  can
49  :  cards
50  :  copy
51  :  new
52  :  cardspdc
53  :  collect
54  :  from
55  :  should
56  :  where
57  :  days
58  :  in
59  :  many
60  :  will
what if i need to know my correct cgpa
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
i have cleared my backlog but the result sheet does not show gpa and cgpa why
[(0, 1), (2, 1), (5, 1), (9, 1), (10, 1), (11, 1),

##Taking input from the user and cleaning the question


In [ ]:
question_orig=input("Enter sentence : ")
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())
print("\n\n",question,"\n",question_embedding)

In [ ]:
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Do you mean: ",FAQdf.iloc[index_sim,0]) 
    print("Answer : ",FAQdf.iloc[index_sim,1])   
    
retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences)

0 0.031234752377721213 what if i need to know my correct cgpa
1 0.031234752377721213 i have cleared my backlog but the result sheet does not show gpa and cgpa why
2 0.031234752377721213 i have cleared my backlog but the result on erp is not updated it still shows earlier cgpaas a result it is affecting my cgpa of next trimesters also how do i get it corrected
3 0.7288480720939872 what do i do if i have not received my grade card
4 0.9079593845004517 my namefathers namemothers name is incorrect on grade card how do i get it corrected
5 0.9079593845004517 how can i get a new copy of corrected grade cards
6 0.9079593845004517 from where should i collect the corrected grade cardspdc
7 0.9079593845004517 in how many days will i get the corrected grade cards


Question:  name is wrong


Do you mean:  My name,Father's Name,Mother's name is incorrect on grade card. How do I get it corrected?
Answer :   Name correction on grade cards/PDC needs to be done in ERP at the office of student section 